In [ ]:
import torch
from torch.utils.benchmark import Timer

# Assuming you have a model and input tensor
model = torch.load('your_model.pth')  # Load your model
input_tensor = torch.randn(1, 3, 224, 224)  # Example input tensor

# Set the model to evaluation mode
model.eval()

# Create a timer
timer = Timer()

# Measure inference time
with torch.no_grad():
    timer.start()
    output = model(input_tensor)
    timer.stop()